In [253]:
import pandas as pd
from pathlib import Path
import numpy as np
import alpaca_trade_api as tradeapi
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

from finta import TA
from finta.utils import resample_calendar
import json
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [254]:
# Alpaca for data
import alpaca_trade_api as api
from alpaca_trade_api.rest import TimeFrame

# pandas for analysis
import pandas as pd

# Plotly for charting
import plotly.graph_objects as go
import plotly.express as px

# Set default charting for pandas to plotly
pd.options.plotting.backend = "plotly"

In [255]:
import datetime as dt
import pytz
#today = dt.date.today()
# symbols we will be looking at
btc = "BTCUSD"
#spy = "SPY"

# start dates and end dates for backtest
#start_date = "2020-01-01"
#end_date = today 
#end_date = '2023-02-06'

end_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(hours=1)).isoformat()

start_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(days=1080)).isoformat()

# time frame for backtests
timeframe = TimeFrame.Day

In [256]:
# Our API keys for Alpaca
API_KEY = "PK5CD8L7ME7T2V9P1IB0"
API_SECRET = "N0bSGJwVRW4jLt0Uhj11DZEXyp7tnKP19FbcCz1P"

# Setup instance of alpaca api
alpaca = api.REST(API_KEY, API_SECRET)

# # # Request historical bar data for SPY and BTC using Alpaca Data API
# for equities, use .get_bars
#spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

# for crypto, use .get_crypto_bars, from multiple exchanges
btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df

btc_data = btc_data[btc_data['exchange'] == 'CBSE']

# display crypto bar data
display(btc_data)
#display(spy_data)

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-02-24 06:00:00+00:00,CBSE,9660.00,9669.99,9532.51,9554.77,1203.778464,12431,9590.793033
2020-02-25 06:00:00+00:00,CBSE,9304.99,9375.36,9092.92,9188.85,4503.851246,24892,9177.637936
2020-02-26 06:00:00+00:00,CBSE,9188.84,9280.00,8609.91,8778.30,15585.993780,79012,8894.716505
2020-02-27 06:00:00+00:00,CBSE,8814.90,8899.94,8779.06,8786.54,1819.421690,15034,8848.335058
2020-02-28 06:00:00+00:00,CBSE,8786.53,8813.12,8428.80,8708.89,11572.123138,63193,8624.928084
...,...,...,...,...,...,...,...,...
2023-02-03 06:00:00+00:00,CBSE,23527.18,23720.00,23208.01,23314.93,13387.860474,364786,23449.094528
2023-02-04 06:00:00+00:00,CBSE,23314.49,23589.67,23233.20,23382.81,3987.582539,232928,23381.304096
2023-02-05 06:00:00+00:00,CBSE,23382.81,23433.20,22637.51,22781.90,8385.600370,261845,22969.010291


In [257]:
ohlcv_df = btc_data.drop(columns=['exchange','trade_count','vwap'])
#ohlcv_df

#ohlcv_df = resample_calendar(ohlcv_df, '4h')
#ohlcv_df = ohlcv_df.dropna()

In [258]:
ohlc_df = ohlcv_df.drop(columns=['volume'])
#ohlc_df

In [259]:
ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 20, kijun_period= 60, senkou_period= 120, chikou_period= 30)
#ichimoku_df

In [260]:
ichimoku_signal_df = pd.concat([ichimoku_df, ohlcv_df['close']], axis=1)
#ichimoku_signal_df

In [261]:

ichimoku_signal_df['signal'] = 0
ichimoku_signal_df["signal"] = np.where((ichimoku_signal_df["TENKAN"] > ichimoku_signal_df["KIJUN"]) &
                                (ichimoku_signal_df["close"] > ichimoku_signal_df["senkou_span_a"]), 1, 0)

ichimoku_signal_df['entry/exit'] = ichimoku_signal_df['signal'].diff()

#ichimoku_signal_df


In [262]:
ichimoku_signal_df['actual_returns'] = ichimoku_signal_df['close'].pct_change()
ichimoku_signal_df["strategy_returns"] = ichimoku_signal_df["actual_returns"] * ichimoku_signal_df["signal"].shift() 


In [263]:
#ichimoku_signal_df

In [264]:
actual_returns = ichimoku_signal_df['actual_returns'].cumsum()

actual_returns_plot =  (1 + ichimoku_signal_df[['actual_returns']]).cumprod().hvplot(
    color='lightblue'
)

In [265]:
strategy_returns_plot = (1 + ichimoku_signal_df[['strategy_returns']]).cumprod().hvplot(
    color='lightgreen'
)

In [266]:
#ichimoku_signal_df

In [267]:
ichimoku_signal_df['entry/exit'].value_counts()

 0.0    1064
 1.0       8
-1.0       7
Name: entry/exit, dtype: int64

In [268]:
entry = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==1]['close'].hvplot.scatter(
    color='green',
    marker= '^'
)

exit = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==-1]['close'].hvplot.scatter(
    color='red',
    marker= 'v'
)

close = ichimoku_signal_df['close'].hvplot(
    color='lightgray',
    
)

#actual_returns = actual_returns.hvplot(
 #   color='lightblue'
#)

#strategy_returns = ichimoku_signal_df['strategy_returns'].hvplot(
 #   color='lightgreen',
#)

In [269]:
close * entry * exit

:Overlay
   .Curve.Close      :Curve   [timestamp]   (close)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [274]:
actual_returns_plot * strategy_returns_plot

:Overlay
   .Curve.I  :Curve   [timestamp]   (actual_returns)
   .Curve.II :Curve   [timestamp]   (strategy_returns)

In [273]:
strategy_returns_annual_volitility = ichimoku_signal_df[['strategy_returns']].std()*np.sqrt(365)
strategy_returns_annual_volitility



strategy_returns    0.456859
dtype: float64

In [281]:
annualized_return = ichimoku_signal_df["strategy_returns"].mean() * 365
annualized_std = ichimoku_signal_df["strategy_returns"].std() * np.sqrt(365)
sharpe_ratio = round(annualized_return/annualized_std, 3)


display(annualized_return)
display(annualized_std)
display(sharpe_ratio)

0.6151766909646756

0.4568588271326144

1.347

In [ ]:
ichimoku_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [ ]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

def get_trends(keywords):
    pytrends.build_payload(kw_list=keywords, timeframe='today 12-m') 
    interest_over_time_data = pytrends.interest_over_time()
    return interest_over_time_data

keywords = ['bitcoin']
trends_data = get_trends(keywords)
print(trends_data)


            bitcoin  isPartial
date                          
2022-02-13       46      False
2022-02-20       62      False
2022-02-27       55      False
2022-03-06       48      False
2022-03-13       42      False
2022-03-20       41      False
2022-03-27       45      False
2022-04-03       38      False
2022-04-10       37      False
2022-04-17       36      False
2022-04-24       34      False
2022-05-01       36      False
2022-05-08       84      False
2022-05-15       52      False
2022-05-22       44      False
2022-05-29       38      False
2022-06-05       39      False
2022-06-12      100      False
2022-06-19       74      False
2022-06-26       54      False
2022-07-03       49      False
2022-07-10       44      False
2022-07-17       46      False
2022-07-24       38      False
2022-07-31       37      False
2022-08-07       35      False
2022-08-14       33      False
2022-08-21       33      False
2022-08-28       33      False
2022-09-04       32      False
2022-09-